In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import json

cols = ['Work_accident','left','promotion_last_5years','sales','salary']
data = pd.read_csv('train.csv',usecols=cols)
data.head()


,Work_accident,left,promotion_last_5years,sales,salary
0,0,1,0,sales,low
1,0,0,0,sales,low
2,0,1,0,accounting,low
3,0,0,0,sales,medium
4,0,0,0,technical,low


In [2]:
print data.shape

(11238, 5)


In [3]:
# for col in data:
#     if col=='left':
#         continue
#     for categ in data[col].unique():
#         print 
#         print col+'  ::  '+str(categ)
#         print
#         print 'Positive : '+str(data.groupby([col])['left'].value_counts()[categ][1])
#         print 'Negative : '+str(data.groupby([col])['left'].value_counts()[categ][0])

In [4]:
def create_tree_node(value, is_leaf):
    new_node = dict()
    new_node['value'] = value
    new_node['is_leaf'] = is_leaf
    new_node['children'] = dict()
    return new_node

In [5]:
def display_tree(t,index):
    print t
    tabs = ''
    for i in range(0,index):
        tabs+='\t'
    print tabs+str(t['value'])
    for c in t['children']:
        i = (index+1)
        display_tree(c,i)
    

In [6]:
def compute_impurity(pos_ratio,neg_ratio,impurity_function):
    if impurity_function == 'entropy':
        if pos_ratio == 0 or pos_ratio == 1:
            return 0
        else:
            return -((pos_ratio*np.log(pos_ratio)) + (neg_ratio*np.log(neg_ratio)))
    elif impurity_function == 'gini_index':
        return (2*pos_ratio*neg_ratio)
    elif impurity_function == 'misclassification_rate':
        return min(pos_ratio,neg_ratio)
    return 0

In [7]:
def print_tree(curr_data,curr_parent_object,curr_condition, exclude_cols,decision_tree,impurity_function):
    
    curr_parent_str = curr_parent_object['value']
    if curr_parent_str not in exclude_cols:
        exclude_cols.append(curr_parent_str)
    #print 'Current Parent object : '+str(curr_parent_object)
    #print 'Current Condition : '+str(curr_condition)
    m = len(curr_data)
    G = dict()
    for col in curr_data:
        if col in exclude_cols:
            continue
        I = 0.0
        col_pos = 0
        col_neg = 0
        for categ in curr_data[col].unique():
            #print 'col :: '+col+' ; categ :: '+str(categ)
            #print curr_data.groupby([col])['left'].value_counts()[categ]
            pos = 0
            neg = 0
            if 1 in curr_data.groupby([col])['left'].value_counts()[categ]: 
                pos = curr_data.groupby([col])['left'].value_counts()[categ][1]
            if 0 in curr_data.groupby([col])['left'].value_counts()[categ]:
                neg = curr_data.groupby([col])['left'].value_counts()[categ][0]
            col_pos += pos
            col_neg += neg
            pos_ratio = float(pos)/float(pos+neg)
            neg_ratio = float(neg)/float(pos+neg)
            impurity = compute_impurity(pos_ratio,neg_ratio, impurity_function)
            I += (float(pos+neg)/float(m))*impurity
        col_pos_ratio = float(col_pos)/float(col_pos+col_neg)
        col_neg_ratio = float(col_neg)/float(col_pos+col_neg)
        E = compute_impurity(col_pos_ratio,col_neg_ratio,impurity_function)
        G[col] = (E-I)
    if not G:
        if curr_parent_str == 'dummy_parent':
            return decision_tree
        for categ in curr_data[col].unique():
            if 0 in curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition] and 1 in curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition]:
                if curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition][0] > curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition][1]:
                    #print 'Left'
                    new_node = create_tree_node('0',True)
                    curr_parent_object['children'][curr_condition] = new_node
                else:
                    new_node = create_tree_node('1',True)
                    curr_parent_object['children'][curr_condition] = new_node
            elif 0 in curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition]:
                new_node = create_tree_node('0',True)
                curr_parent_object['children'][curr_condition] = new_node
            elif 1 in curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition]:
                new_node = create_tree_node('1',True)
                curr_parent_object['children'][curr_condition] = new_node
            else:
                new_node = create_tree_node('0',True)
                curr_parent_object['children'][curr_condition] = new_node
        return decision_tree
    
    max_col = max(G, key=G.get)
    
    new_node = create_tree_node(max_col,False)
    if decision_tree is None:
        decision_tree = new_node
    else:
        curr_parent_object['children'][curr_condition] = new_node
    
    res_list = list()
    next_conds = list()
    for cond in curr_data[max_col].unique():
        if 0 in curr_data.groupby([max_col])['left'].value_counts()[cond]:
            res_list.append(curr_data.groupby([max_col])['left'].value_counts()[cond][0])
        else:
            res_list.append(0)
        if 1 in curr_data.groupby([max_col])['left'].value_counts()[cond]:
            res_list.append(curr_data.groupby([max_col])['left'].value_counts()[cond][1])
        else:
            res_list.append(0)
        res_sum = sum(res_list)
        if res_sum == 0:
            new_leaf_node = create_tree_node('0',True)
            new_node['children'][cond] = new_leaf_node
        else:
            ratio = max(res_list)/res_sum
            if ratio > 0.90:
                if res_list[0] == max(res_list):
                    new_leaf_node = create_tree_node('0',True)
                    new_node['children'][cond] = new_leaf_node
                else:
                    new_leaf_node = create_tree_node('1',True)
                    new_node['children'][cond] = new_leaf_node
            else:
                next_conds.append(cond)
    
    
    
    temp_exclude_cols = exclude_cols[:]
    for cond in next_conds:
        print_tree(curr_data.loc[curr_data[max_col]==cond].copy() , new_node, cond,temp_exclude_cols,decision_tree,impurity_function)
    
    return decision_tree

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    data,
    data.left,
    test_size=0.2,
    random_state=0)

decision_tree_entropy = None
decision_tree_gini = None
decision_tree_mr = None
dummy_node = dict()
dummy_node['value'] = 'dummy_parent'
decision_tree_entropy = print_tree(X_train.copy(),dummy_node,'dummy_condition',['left'],decision_tree_entropy,'entropy')
decision_tree_gini = print_tree(X_train.copy(),dummy_node,'dummy_condition',['left'],decision_tree_gini,'gini_index')
decision_tree_mr = print_tree(X_train.copy(),dummy_node,'dummy_condition',['left'],decision_tree_mr,'misclassification_rate')
# print json.dumps(decision_tree)

In [9]:
def validate_row(row,tree_node):
    if tree_node['is_leaf']:
        if row['left'] == 1 and tree_node['value'] == '1':
            #true positive
            return 2
        elif row['left'] == 1 and tree_node['value'] == '0':
            #false negative
            return -1
        elif row['left'] == 0 and tree_node['value'] == '1':
            #false positive
            return 0
        elif row['left'] == 0 and tree_node['value'] == '0':
            #true negative
            return 1
    key = tree_node['value']
    val = row[key]
    if val not in tree_node['children']:
        #error considered as true negative
        return 1
    else:
        return validate_row(row,tree_node['children'][val])

In [10]:
def display_performance_metrics(X,tree):
    m = len(X)
    hits = 0
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for index,row in X.iterrows():
        verdict = validate_row(row,tree)
        if verdict == -1:
            fn += 1
        elif verdict == 0:
            fp += 1
        elif verdict == 1:
            hits += 1
            tn += 1
        elif verdict == 2:
            hits += 1
            tp += 1
    
    accuracy = float(hits)/float(m)
    misclassification = 1.0 - accuracy
    print 'Accuracy : '+str(accuracy*100)
    print 'Misclassification error : '+str(misclassification * 100)
    precision = float(tp)/(float(tp)+float(fp))
    recall = float(tp)/(float(tp)+float(fn))
    print 'Precision : '+str(precision)
    print 'Recall : '+str(recall)

    f1_score = 2.0/((1.0/recall)+(1.0/precision))
    print 'F1 score : '+str(f1_score)

In [11]:
print '************TRAIN PERFORMANCE*************'
print
print '::::Impurity Function : Entropy::::'
print
display_performance_metrics(X_train,decision_tree_entropy)
print '******************'
print
print '::::Impurity Function : Gini Index::::'
print
display_performance_metrics(X_train,decision_tree_gini)
print '******************'
print
print '::::Impurity Function : Misclassification rate::::'
print
display_performance_metrics(X_train,decision_tree_mr)
print '******************'

************TRAIN PERFORMANCE*************

::::Impurity Function : Entropy::::

Accuracy : 76.1624026696
Misclassification error : 23.8375973304
Precision : 0.75
Recall : 0.00279459711225
F1 score : 0.00556844547564
******************

::::Impurity Function : Gini Index::::

Accuracy : 76.1624026696
Misclassification error : 23.8375973304
Precision : 0.75
Recall : 0.00279459711225
F1 score : 0.00556844547564
******************

::::Impurity Function : Misclassification rate::::

Accuracy : 76.1624026696
Misclassification error : 23.8375973304
Precision : 0.75
Recall : 0.00279459711225
F1 score : 0.00556844547564
******************


In [12]:
print '************TEST PERFORMANCE*************'
print
print '::::Impurity Function : Entropy::::'
print
display_performance_metrics(X_test,decision_tree_entropy)
print '******************'
print
print '::::Impurity Function : Gini Index::::'
print
display_performance_metrics(X_test,decision_tree_gini)
print '******************'
print
print '::::Impurity Function : Misclassification rate::::'
print
display_performance_metrics(X_test,decision_tree_mr)
print '******************'

************TEST PERFORMANCE*************

::::Impurity Function : Entropy::::

Accuracy : 76.5569395018
Misclassification error : 23.4430604982
Precision : 1.0
Recall : 0.00189393939394
F1 score : 0.00378071833648
******************

::::Impurity Function : Gini Index::::

Accuracy : 76.5569395018
Misclassification error : 23.4430604982
Precision : 1.0
Recall : 0.00189393939394
F1 score : 0.00378071833648
******************

::::Impurity Function : Misclassification rate::::

Accuracy : 76.5569395018
Misclassification error : 23.4430604982
Precision : 1.0
Recall : 0.00189393939394
F1 score : 0.00378071833648
******************
